In [1]:
import sys
import keras
import numpy as np
from keras import layers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegVectorizer
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print(vec_length)

252


### Standard autoencoder

Design the network

In [23]:
encoding_dim = 100
input_reg = keras.Input(shape=(vec_length,))
encoded = layers.Dense(encoding_dim, activation='relu')(input_reg)
decoded = layers.Dense(vec_length, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_reg, decoded)
encoder = keras.Model(input_reg, encoded)
encoded_input = keras.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

Do some learning

In [24]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(train_vecs, train_vecs,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(test_vecs, test_vecs))

Epoch 1/50
40/40 [==============================] - 1s 16ms/step - loss: 0.5471 - val_loss: 0.2914
Epoch 2/50
40/40 [==============================] - 0s 11ms/step - loss: 0.1644 - val_loss: 0.1191
Epoch 3/50
40/40 [==============================] - 0s 12ms/step - loss: 0.1144 - val_loss: 0.1109
Epoch 4/50
40/40 [==============================] - 0s 12ms/step - loss: 0.1088 - val_loss: 0.1069
Epoch 5/50
40/40 [==============================] - 0s 11ms/step - loss: 0.1053 - val_loss: 0.1037
Epoch 6/50
40/40 [==============================] - 0s 12ms/step - loss: 0.1021 - val_loss: 0.1007
Epoch 7/50
40/40 [==============================] - 0s 12ms/step - loss: 0.0990 - val_loss: 0.0975
Epoch 8/50
40/40 [==============================] - 0s 12ms/step - loss: 0.0956 - val_loss: 0.0940
Epoch 9/50
40/40 [==============================] - 1s 15ms/step - loss: 0.0920 - val_loss: 0.0903
Epoch 10/50
40/40 [==============================] - 1s 15ms/step - loss: 0.0880 - val_loss: 0.0862
Epoch 11/

In [25]:
encoded_regs = encoder.predict(test_vecs[:10])
decoded_regs = decoder.predict(encoded_regs)
print([v.recover(x) for x in test_vecs[:10]])

['AN45FVI', 'LE23DCR', 'HA99JWO', 'EQ58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']


In [26]:
print([v.recover(x) for x in decoded_regs])

['AN45FVI', 'LE23DCR', 'HA99JWO', 'EQ58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']
